###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2021 Semester 1

## Assignment 2: Duration Classification with Recipe Data


**Student ID(s): 1004503, 1005418

## Import required libraries

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#import pickle
import scipy
import gc
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier
from collections import defaultdict
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

## Pre-process for count vectorizer and doc2vec

In [8]:
# Count Vectoriser process

count_vec_name = scipy.sparse.load_npz("ML2_data/recipe_text_features_countvec/train_name_vec.npz").toarray()
#count_vec_steps = scipy.sparse.load_npz("ML2_data/recipe_text_features_countvec/train_steps_vec.npz").toarray()
#count_vec_ingr = scipy.sparse.load_npz("ML2_data/recipe_text_features_countvec/train_ingr_vec.npz").toarray()

In [9]:
# Doc2vec process

doc2vec100_name = pd.read_csv("ML2_data/recipe_text_features_doc2vec100/train_name_doc2vec100.csv", header = None).values
doc2vec100_steps = pd.read_csv("ML2_data/recipe_text_features_doc2vec100/train_steps_doc2vec100.csv", header = None).values
doc2vec100_ingr = pd.read_csv("ML2_data/recipe_text_features_doc2vec100/train_ingr_doc2vec100.csv", header = None).values

## Extract data from recipe csv

In [10]:
# Isolate duration label
csv_data = pd.read_csv("ML2_data/recipe_train.csv")

n_steps = csv_data.iloc[:,1]
n_steps = n_steps.values

n_ingr = csv_data.iloc[:,2]
n_ingr = n_ingr.values

duration_array = csv_data.iloc[:,-1]
duration_array = duration_array.values

n_steps_array = []
n_ingr_array = []
for i in range(len(n_steps)):
    n_steps_array.append([n_steps[i]])
    n_ingr_array.append([n_ingr[i]])

In [11]:
train_data = np.hstack([n_steps_array,n_ingr_array,doc2vec100_name,doc2vec100_steps,doc2vec100_ingr,count_vec_name])
print(train_data.shape)

X_train, X_test, y_train, y_test = train_test_split(train_data, duration_array, test_size=0.25, random_state=42)

selection = SelectKBest(score_func=f_classif, k=1000)
selection.fit(X_train,y_train)
X_train = selection.transform(X_train)
X_test = selection.transform(X_test)

del count_vec_name
del doc2vec100_name 
del doc2vec100_steps
del doc2vec100_ingr 
del n_steps
del n_ingr
del duration_array
del n_steps_array 
del n_ingr_array

gc.collect()

(40000, 11194)


MemoryError: Unable to allocate 1.11 GiB for an array with shape (13305, 11194) and data type float64

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn import svm
import time

#try different classifiers and evaluate individual model performance
#code adapated from Practical 8

models = [DummyClassifier(strategy='most_frequent'),
          LogisticRegression(),
          GaussianNB(),
          MultinomialNB(),
          svm.LinearSVC(),
          DecisionTreeClassifier(),
          KNeighborsClassifier()]

titles = ['0-R',
          'Logistic Regression',
          'GNB',
          'MNB',
          'LinearSVC',
          'Decision Tree',
          'KNN']

# for title, model in zip(titles, models):
    
#     output = []
#     model.fit(X_train, y_train)
#     y_predict = model.predict(X_test)
    
#     for i in range(len(y_predict)):
#         row = [i+1, y_predict[i]]
#         output.append(row)
    
    
#      output = pd.DataFrame(output, columns = ['id', 'duration_label'])
#      output.to_csv(title + '_CV.csv', index = False)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
print(clf.score(X_test, y_test))

xgb = XGBClassifier(random_state=0).fit(X_train, y_train)
print(xgb.score(X_test, y_test))

# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))


0.7461
[22:48:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.7361


# Havent refactored under here


In [8]:
from sklearn.metrics import accuracy_score

#code excerpted from Practical 8
#creates a stacking classifier class given a set of classifiers and a meta-classifier
class StackingClassifier():

    def __init__(self, classifiers, metaclassifier):
        self.classifiers = classifiers
        self.metaclassifier = metaclassifier

    def fit(self, X, y):
        for clf in self.classifiers:
            clf.fit(X, y)
        X_meta = self._predict_base(X)
        self.metaclassifier.fit(X_meta, y)
    
    def _predict_base(self, X):
        yhats = []
        for clf in self.classifiers:
            yhat = clf.predict_proba(X)
            yhats.append(yhat)
        yhats = np.concatenate(yhats, axis=1)
        assert yhats.shape[0] == X.shape[0]
        return yhats
    
    def predict(self, X):
        X_meta = self._predict_base(X)     
        yhat = self.metaclassifier.predict(X_meta)
        return yhat
    def score(self, X, y):
        yhat = self.predict(X)
        return accuracy_score(y, yhat)

In [13]:
#initialise a meta classifier with Logistic Regression as the meta classifier

classifiers = [LogisticRegression(),
          GaussianNB(),
          MultinomialNB(),
          DecisionTreeClassifier(),
          KNeighborsClassifier()]

meta_clf_lr = LogisticRegression()
stacker_lr = StackingClassifier(classifiers, meta_clf_lr)

#create stacked model output
output = []
stacker_lr.fit(Xtrain, y_train)
y_predict = stacker_lr.predict(Xtest)
    
for i in range(len(y_predict)):
    row = [i+1, y_predict[i]]
    output.append(row)
    
output = pd.DataFrame(output, columns = ['id', 'duration_label'])
output.to_csv('stacker' + '_CV.csv', index = False)